### ollama本地部署大模型

- 首先前往[官网](https://ollama.com/)下载软件并安装
  
- 在windows系统搜索框搜索`终端`并打开
  
- 接着可以在[模型列表](https://ollama.com/search)中找想要的下载的模型，复制下载命令即可，例如我下载qwen2.5-7b的模型：`ollama pull qwen2.5`，这里会默认下载7b的。若下载过程中卡住，则用键盘点击 ctrl + c 终止下载，然后再运行下载命令即可
  
- 你还可以测试模型是否能正常运行：`ollama run qwen2.5`
- 后续运行大模型时，需要确保ollama运用程序已开启，可以查看windows右下角的隐藏图标。若未看到ollama图标，可以在搜索框搜索ollama并点击运行

In [5]:
import json
from typing import List, Dict, Any
from langchain_ollama import ChatOllama
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, BaseMessage

In [6]:
# ------------------ 初始化模型 ------------------
llm = ChatOllama(model="qwen2.5", temperature=0)  # temperature=0 保证输出稳定

In [7]:
# ------------------ 通用调用封装 ------------------
def run_chain(history: List[BaseMessage]) -> Dict[str, Any]:
    """
    统一调用 LLM 并返回结构化结果
    :param history: 包含 system/user/ai 的历史消息
    :return: LLM 返回的 JSON 字符串解析后的 dict
    """
    response = llm.invoke(history)
    response.pretty_print()  
    try:
        return json.loads(response.content)  # 解析 JSON，方便下游使用
    except Exception as e:
        return {"error": str(e), "raw": response.content}

In [8]:
# ------------------ 示例 1：情绪抽取 ------------------
emotion_schema = {"emotion": "开心 | 生气 | 悲伤 | 惊讶 | 恐惧 | 厌恶 | 中性"}
emotion_examples = [
    ("今天发工资啦，晚饭吃火锅庆祝！", "开心"),
    ("老板又临时加需求，真受不了！", "生气"),
    ("我的狗走丢了，已经三天没回家。", "悲伤"),
]


def build_emotion_prompt(target: str) -> List[BaseMessage]:
    """构造情绪抽取的 few-shot prompt"""
    messages = [
        SystemMessage(
            "请从句子中提取用户的情绪（emotion），并以 JSON 形式返回，例如：\n"
            f"{emotion_schema}"
        )
    ]
    for sent, label in emotion_examples:
        messages.extend([HumanMessage(sent), AIMessage(f'{{"emotion": "{label}"}}')])
    messages.append(HumanMessage(target))
    return messages


print("【示例 1：情绪抽取】")
emotion_result = run_chain(build_emotion_prompt("没想到电影结局这么反转，完全没猜到！"))
print("解析结果：", emotion_result, "\n")

【示例 1：情绪抽取】
================================== Ai Message ==================================

{"emotion": "惊讶"}
解析结果： {'emotion': '惊讶'} 



In [9]:
# ------------------ 示例 2：人物信息抽取 ------------------
person_schema = {"name": "姓名", "hair_color": "发色", "height_cm": "身高(cm)"}
person_examples = [
    (
        "李雷身高180厘米，头发是棕色的。",
        {"name": "李雷", "hair_color": "棕色", "height_cm": 180},
    ),
    (
        "金发的韩梅梅只有158cm高。",
        {"name": "韩梅梅", "hair_color": "金色", "height_cm": 158},
    ),
    (
        "王芳染了一头红发，身高约165厘米。",
        {"name": "王芳", "hair_color": "红色", "height_cm": 165},
    ),
]


def build_person_prompt(target: str) -> List[BaseMessage]:
    """构造人物信息抽取的 few-shot prompt"""
    messages = [
        SystemMessage(
            "请从句子中提取人物姓名、发色（hair_color）和身高（height_cm），"
            "并以 JSON 形式返回，例如：\n"
            f"{person_schema}"
        )
    ]
    for sent, info in person_examples:
        messages.extend(
            [
                HumanMessage(sent),
                AIMessage(str(info).replace("'", '"')),  # 转双引号
            ]
        )
    messages.append(HumanMessage(target))
    return messages


print("【示例 2：人物信息抽取】")
person_result = run_chain(
    build_person_prompt("张小明是黑发，据说他现在已经长到172厘米了。")
)
print("解析结果：", person_result)

【示例 2：人物信息抽取】
================================== Ai Message ==================================

{"name": "张小明", "hair_color": "黑色", "height_cm": 172}
解析结果： {'name': '张小明', 'hair_color': '黑色', 'height_cm': 172}
